In [1]:
'''
#첫번째 레이어
합성곱(in_channel = 1, out_channel = 32, kernel_size = 3, stride = 1, padding = 1) + 활성화 함수(activation function) ReLU
맥스풀링(kernel_size = 2, stride = 2)

#두번째 레이어
합성곱(in_channel = 23, out_channel = 64, kernel_size = 3, stride = 1, padding = 1) + 활성화 함수(activation function) ReLU
맥스풀링(kernel_size = 2, stride = 2)

#세번째 레이어
batch_size * 7 * 7 * 64 => batch size = 3136 (특성맵을 펼치는 레이어)
Fully connected(전결합층 뉴런 10개) + 활성화 함수 softmax
'''

'\n#첫번째 레이어\n합성곱(in_channel = 1, out_channel = 32, kernel_size = 3, stride = 1, padding = 1) + 활성화 함수(activation function) ReLU\n맥스풀링(kernel_size = 2, stride = 2)\n\n#두번째 레이어\n합성곱(in_channel = 23, out_channel = 64, kernel_size = 3, stride = 1, padding = 1) + 활성화 함수(activation function) ReLU\n맥스풀링(kernel_size = 2, stride = 2)\n\n#세번째 레이어\nbatch_size * 7 * 7 * 64 => batch size = 3136 (특성맵을 펼치는 레이어)\nFully connected(전결합층 뉴런 10개) + 활성화 함수 softmax\n'

In [13]:
import torch
import torch.nn as nn

In [14]:
#임의의 텐서 생성 (1,1,28,28) -> MNIST데이터
inputs = torch.Tensor(1,1,28,28)
print('텐서의 크기: {}'.format(inputs.shape))

텐서의 크기: torch.Size([1, 1, 28, 28])


In [15]:
#첫번째 합성곱층 선언
conv1 = nn.Conv2d(1, 32, 3, padding = 1)   #Conv2d내의 인자값(input_channel,output_channel,커널 사이즈,패딩)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [23]:
#두번째 합성곱층 선언
conv2 = nn.Conv2d(32, 64, 3, padding = 1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [24]:
#세번째 합성곱층 선언(풀링 레이어)
pool = nn.MaxPool2d(2)   #간단히 선언하면 선언된 하나의 정수값을 통해 kernel size, stride가 결정되게 된다
print(pool)


MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [25]:
#선언된 레이어를 연결하는 과정
#첫번째 레이어 지나가기
out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


In [26]:
#컨벌루셔널 레이어를 거쳐갈 경우 마지막에 pooling layer가 존재
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [27]:
#두번째 레이어 지나가기
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


In [28]:
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [29]:
#view() 함수를 사용해 텐서를 펼치는 작업
out = out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


In [30]:
fc = nn.Linear(3136,10)   #fully connected:전체를 연결(기초적인 신경망), 통합된 텐서를 통해 0~9까지의 결과를 갖기땜에 10개의 output
out=fc(out)
print(out.shape)

#여기가까지가 전체 모델 설정 과정

torch.Size([1, 10])


In [31]:
#위에서 선언한 신경망을 통해 MNIST(숫자 데이터셋) 분류문제 해결
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [34]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#torch.cuda.is_available()함수는 True/False의 boolean형태로 나타나게되는데, CUDA를 사용 가능할 경우 True반환하기 때문에
#if 조건을 만족하는 device='cuda'가 되고 사용불가 시 False를 반환하기 때문에 else조건을 만족하는 device='cpu'가 됨

#GPU 사용시에 고정시킬 시드 선언
torch.manual_seed(777)

#만약 CPU가 아닌 GPU를 사용해 훈련을 진행할 경우 고정된 시드를 통해 GPU 로 학습
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [36]:
#학습에 사용되는 변수(파라미터들 학습률, 에폭, 배치사이즈 등) 선언
learning_rate=0.001
training_epoch=15
batch_size=100

In [37]:
#MNIST train, test 데이터셋을 torchvision.datasets를 통해 비교적 쉽게 가져오기
mnist_train = dsets.MNIST(root='MNIST_data/', train=True, transform=transforms.ToTensor(), download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', train=False, transform=transforms.ToTensor(), download=True)


0.0%

31.0%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

91.1%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [38]:
#데이터로더를 이용해 배치크기 결정
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size,shuffle=True,drop_last=True)

In [41]:
class CNN(torch.nn.Module):   #CNN구조를 사용한 모델을 선언하기 위한 클래스
    def __init__(self):   #초기화 함수, 동시에 모델의 구조를 선언하는 함수
        super(CNN, self).__init__()
        
        self.layer1 = torch.nn.Sequential(torch.nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1), torch.nn.ReLU(),
                                          torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = torch.nn.Sequential(torch.nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1), torch.nn.ReLU(),
                                          torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.fc = torch.nn.Linear(7*7*64, 10, bias=True)
        
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self,x):   #레이어를 거치며 계산되는 가중치를 전달하기 위해 사용하는 함수
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0),-1)
        out = self.fc(out)
        
        return out

In [42]:
model = CNN().to(device)   #model 이라는 객체를 통해 CNN클래스가 선언이 되고 이는 .to(device)를 통해 GPU에서 연산을 하게됨
  

In [44]:
# 배용함수(loss function = cost function)은 크로스엔트로피로 설정
#옵티마이저는 Adam으로 설정
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [47]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))
#MNIST dataset은 60000개의 데이터를 가지고 있는데 batch_size를 100번으로 설정했으므로 총 600의 batch_size를 가지게됨

총 배치의 수 : 600


In [50]:
for epoch in range(training_epoch):
    avg_cost = 0
    
    for X,Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch+1, avg_cost))


[Epoch:    1] cost = 0.0051192441
[Epoch:    2] cost = 0.00445792684
[Epoch:    3] cost = 0.00460245507
[Epoch:    4] cost = 0.00304334541
[Epoch:    5] cost = 0.00230825413
[Epoch:    6] cost = 0.00578143727
[Epoch:    7] cost = 0.00336918654
[Epoch:    8] cost = 0.00140294537
[Epoch:    9] cost = 0.00310330023
[Epoch:   10] cost = 0.0025977646
[Epoch:   11] cost = 0.00306333415
[Epoch:   12] cost = 0.00145115319
[Epoch:   13] cost = 0.00251600705
[Epoch:   14] cost = 0.00151396764
[Epoch:   15] cost = 0.0016085048


In [51]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9835999608039856


C:\Users\03\anaconda3\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\03\anaconda3\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
